In [ ]:
import pandas as pd

dataset_splits = {'train': 'Dataset_IMDB_Sentiment/plain_text/train-00000-of-00001.parquet', 'test': 'Dataset_IMDB_Sentiment/plain_text/test-00000-of-00001.parquet', 'unsupervised': 'Dataset_IMDB_Sentiment/plain_text/unsupervised-00000-of-00001.parquet'}
imdb_train_df = pd.read_parquet(dataset_splits["train"])

In [9]:
print(df.head(5))

                                                text  label
0  I rented I AM CURIOUS-YELLOW from my video sto...      0
1  "I Am Curious: Yellow" is a risible and preten...      0
2  If only to avoid making this type of film in t...      0
3  This film was probably inspired by Godard's Ma...      0
4  Oh, brother...after hearing about this ridicul...      0
